In [ ]:
import os, math, random, json
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

In [ ]:
#reproducibility & device
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

Device: cpu


In [ ]:
# Synthetic multivariate generator
def generate_synthetic_multivariate(T=5000, n_series=3, noise_std=0.25):
    t = np.arange(T)
    trend = 0.00002 * (t - T / 2) ** 2 + 0.00015 * t
    s1 = 2.0 * np.sin(2 * np.pi * t / 50.0)
    s2 = 0.9 * np.cos(2 * np.pi * t / 200.0 + 0.3)
    s3 = 0.4 * np.sin(2 * np.pi * t / 7.0)
    base = trend + s1 + s2 + s3

    temp = 10 + 8 * np.sin(2 * np.pi * t / 365.0) + 0.6 * np.sin(2 * np.pi * t / 30.0)
    holiday = (signal.square(2 * np.pi * t / 180.0, duty=0.02) + 1) / 2
    exog = np.stack([temp, holiday], axis=1).astype(np.float32)

    X = np.zeros((T, n_series), dtype=np.float32)
    for i in range(n_series):
        a = 1.0 + 0.2 * i
        b_temp = 0.05 * (i + 1)
        b_hol = 0.8 * (0.5 if i == 0 else (0.3 * (i + 1)))
        kernel = np.exp(-np.linspace(0, 3, 30) * (0.5 + 0.15 * i))
        interaction = np.convolve(base, kernel, mode='same')[:T]
        series = a * interaction + b_temp * temp + b_hol * holiday
        noise = noise_std * (1 + 0.4 * np.sin(2 * np.pi * t / 120.0)) * np.random.randn(T)
        X[:, i] = series + noise
    return X.astype(np.float32), exog

In [ ]:
# Dataset
class Seq2SeqDataset(Dataset):
    def __init__(self, data, exog, input_len, output_len, scaler=None):
        assert data.shape[0] == exog.shape[0]
        self.data = data
        self.exog = exog
        self.input_len = input_len
        self.output_len = output_len
        self.T = data.shape[0]
        self.n_series = data.shape[1]
        self.n_exog = exog.shape[1]

        if scaler is None:
            self.scaler = StandardScaler()
            self.scaler.fit(self.data)
        else:
            self.scaler = scaler
        self.data_scaled = self.scaler.transform(self.data)

        self.starts = list(range(0, self.T - (input_len + output_len) + 1))

    def __len__(self):
        return len(self.starts)

    def __getitem__(self, idx):
        s = self.starts[idx]
        x = self.data_scaled[s:s+self.input_len]            # (input_len, n_series)
        y = self.data_scaled[s+self.input_len:s+self.input_len+self.output_len]  # (output_len, n_series)
        ex = self.exog[s:s+self.input_len+self.output_len]  # (input_len+output_len, n_exog)
        return {
            'x': torch.tensor(x, dtype=torch.float32),
            'y': torch.tensor(y, dtype=torch.float32),
            'ex': torch.tensor(ex, dtype=torch.float32)
        }

In [ ]:
# Positional encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=20000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer('pe', pe)
    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]

In [ ]:
# Attention / FF / Layers
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        assert d_model % n_heads == 0
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads

        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.out = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, v, mask=None):
        B = q.size(0)
        Q = self.q_linear(q).view(B, -1, self.n_heads, self.d_k).transpose(1,2)
        K = self.k_linear(k).view(B, -1, self.n_heads, self.d_k).transpose(1,2)
        V = self.v_linear(v).view(B, -1, self.n_heads, self.d_k).transpose(1,2)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        attn = torch.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        context = torch.matmul(attn, V)
        context = context.transpose(1,2).contiguous().view(B, -1, self.d_model)
        out = self.out(context)
        return out, attn

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model)
        )
    def forward(self, x):
        return self.net(x)

class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadSelfAttention(d_model, n_heads, dropout)
        self.ff = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.drop = nn.Dropout(dropout)
    def forward(self, x, src_mask=None):
        attn_out, attn_w = self.self_attn(x, x, x, mask=src_mask)
        x = self.norm1(x + self.drop(attn_out))
        ff = self.ff(x)
        x = self.norm2(x + self.drop(ff))
        return x, attn_w

class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadSelfAttention(d_model, n_heads, dropout)
        self.cross_attn = MultiHeadSelfAttention(d_model, n_heads, dropout)
        self.ff = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.drop = nn.Dropout(dropout)
    def forward(self, x, enc_out, src_mask=None, tgt_mask=None):
        attn_out, self_w = self.self_attn(x, x, x, mask=tgt_mask)
        x = self.norm1(x + self.drop(attn_out))
        cross_out, cross_w = self.cross_attn(x, enc_out, enc_out, mask=src_mask)
        x = self.norm2(x + self.drop(cross_out))
        ff = self.ff(x)
        x = self.norm3(x + self.drop(ff))
        return x, self_w, cross_w

In [ ]:
# Transformer Seq2Seq
class TransformerTimeSeries(nn.Module):
    def __init__(self, n_series, n_exog, d_model=128, n_heads=4, n_enc_layers=3, n_dec_layers=3, d_ff=256, dropout=0.1):
        super().__init__()
        self.input_proj = nn.Linear(n_series + n_exog, d_model)
        self.output_proj = nn.Linear(d_model, n_series)
        self.pos_enc = PositionalEncoding(d_model)
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_enc_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_dec_layers)])
    def forward(self, src_series, src_exog, tgt_series, tgt_exog, src_mask=None, tgt_mask=None):
        batch = src_series.size(0)
        src_len = src_series.size(1)
        tgt_len = tgt_series.size(1)

        src_ex = src_exog[:, :src_len, :]
        tgt_ex = tgt_exog[:, src_len:src_len+tgt_len, :]

        enc_in = torch.cat([src_series, src_ex], dim=-1)
        dec_in = torch.cat([tgt_series, tgt_ex], dim=-1)

        enc = self.input_proj(enc_in)
        dec = self.input_proj(dec_in)

        enc = self.pos_enc(enc)
        dec = self.pos_enc(dec)

        enc_attns = []
        for layer in self.encoder_layers:
            enc, attw = layer(enc, src_mask)
            enc_attns.append(attw)

        dec_self_attns = []
        dec_cross_attns = []
        for layer in self.decoder_layers:
            dec, self_w, cross_w = layer(dec, enc, src_mask, tgt_mask)
            dec_self_attns.append(self_w)
            dec_cross_attns.append(cross_w)

        out = self.output_proj(dec)
        return out, {'enc_attns': enc_attns, 'dec_self_attns': dec_self_attns, 'dec_cross_attns': dec_cross_attns}

In [ ]:
# Utility: masks & metrics
def create_causal_mask(tgt_len, device):
    # returns boolean mask shape (1,1,tgt_len,tgt_len)
    tri = torch.tril(torch.ones((tgt_len, tgt_len), dtype=torch.bool, device=device))
    return tri

def rmse(pred, true):
    return float(np.sqrt(np.mean((pred - true)**2)))

def mae(pred, true):
    return float(np.mean(np.abs(pred - true)))

def mape(pred, true):
    eps = 1e-8
    return float(np.mean(np.abs((true - pred) / (np.abs(true) + eps)))) * 100.0

In [ ]:
# Evaluation
def evaluate_model(model, dataloader, scaler, device):
    model.eval()
    preds = []
    trues = []
    with torch.no_grad():
        for batch in dataloader:
            x = batch['x'].to(device)
            y = batch['y'].to(device)
            ex = batch['ex'].to(device)
            batch_size, tgt_len, n_series = y.size()
            dec_in = torch.zeros((batch_size, tgt_len, n_series), device=device)
            out, _ = model(x, ex, dec_in, ex)
            if isinstance(out, tuple) or isinstance(out, list):
                out = out[0]
            out_np = out.cpu().numpy()
            y_np = y.cpu().numpy()
            out_un = scaler.inverse_transform(out_np.reshape(-1, n_series)).reshape(out_np.shape)
            y_un = scaler.inverse_transform(y_np.reshape(-1, n_series)).reshape(y_np.shape)
            preds.append(out_un)
            trues.append(y_un)
    preds = np.concatenate(preds, axis=0)
    trues = np.concatenate(trues, axis=0)
    return preds, trues

In [ ]:
# Training Loop
def train_model(model, train_loader, val_loader, scaler, epochs=12, lr=3e-4, device=DEVICE, save_path="best_model.pt"):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    best_val = float('inf')
    best_path = save_path

    print("Training started...\n")
    for epoch in range(1, epochs+1):
        model.train()
        total_loss = 0.0
        for batch in train_loader:
            x = batch['x'].to(device)
            y = batch['y'].to(device)
            ex = batch['ex'].to(device)
            optimizer.zero_grad()
            dec_in = torch.cat([torch.zeros_like(y[:, :1, :]), y[:, :-1, :]], dim=1)
            out, _ = model(x, ex, dec_in, ex)
            if isinstance(out, (tuple, list)):
                out = out[0]
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * x.size(0)
        avg_loss = total_loss / len(train_loader.dataset)
        preds_val, trues_val = evaluate_model(model, val_loader, scaler, device)
        val_rmse = rmse(preds_val, trues_val)
        if val_rmse < best_val:
            best_val = val_rmse
            torch.save(model.state_dict(), best_path)
        print(f"Epoch {epoch}/{epochs} | Train Loss: {avg_loss:.6f} | Val RMSE: {val_rmse:.6f}")
    print("\nTraining complete! Best model saved to:", best_path)
    model.load_state_dict(torch.load(best_path, map_location=device))
    return model, best_path

In [ ]:
# Attention plot
def plot_attention_weights(attn_weights, title="Attention", savepath=None):
    if not attn_weights:
        print("No attention weights.")
        return
    avg_maps = []
    for layer_w in attn_weights:
        w = layer_w.detach().cpu().numpy()
        avg = w.mean(axis=0).mean(axis=0)
        avg_maps.append(avg)
    n = len(avg_maps)
    fig, axes = plt.subplots(1, n, figsize=(4*n,4))
    if n==1:
        axes=[axes]
    for i,ax in enumerate(axes):
        im = ax.imshow(avg_maps[i], aspect='auto')
        ax.set_title(f"Layer {i}")
        fig.colorbar(im, ax=ax)
    plt.suptitle(title)
    if savepath:
        plt.savefig(savepath)
        print("Saved attention plot to", savepath)
    else:
        plt.show()
    plt.close()

In [ ]:
# Main (run)
def main():
    os.makedirs("outputs", exist_ok=True)

    # Data
    T = 5000
    X, exog = generate_synthetic_multivariate(T=T, n_series=3)
    print("Data shapes:", X.shape, exog.shape)

    # Splits
    train_T = int(0.7*T)
    val_T = int(0.85*T)

    scaler = StandardScaler().fit(X[:train_T])

    # Hyperparams
    input_len = 96
    output_len = 48
    batch_size = 128

    train_ds = Seq2SeqDataset(X[:train_T], exog[:train_T], input_len, output_len, scaler=scaler)
    val_ds = Seq2SeqDataset(X[train_T:val_T], exog[train_T:val_T], input_len, output_len, scaler=scaler)
    test_ds = Seq2SeqDataset(X[val_T:], exog[val_T:], input_len, output_len, scaler=scaler)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

    # Model
    model = TransformerTimeSeries(n_series=3, n_exog=exog.shape[1], d_model=128, n_heads=4, n_enc_layers=3, n_dec_layers=3, d_ff=256).to(DEVICE)

    # Train
    model, best_path = train_model(model, train_loader, val_loader, scaler, epochs=12, lr=3e-4, device=DEVICE, save_path="outputs/best_model.pt")

    # Evaluate on test
    preds, trues = evaluate_model(model, test_loader, scaler, DEVICE)
    print("Test RMSE:", rmse(preds, trues), "MAE:", mae(preds, trues), "MAPE:", mape(preds, trues))

    # Attention viz (one batch)
    sample = next(iter(test_loader))
    x = sample['x'].to(DEVICE); y = sample['y'].to(DEVICE); ex = sample['ex'].to(DEVICE)
    dec_in = torch.cat([torch.zeros_like(y[:,:1,:]), y[:, :-1, :]], dim=1)
    model.eval()
    with torch.no_grad():
        out, attn = model(x, ex, dec_in, ex)
    attn_save = "outputs/dec_cross_attn.png"
    if attn and 'dec_cross_attns' in attn and len(attn['dec_cross_attns'])>0:
        plot_attention_weights(attn['dec_cross_attns'], title="Decoder Cross-Attention", savepath=attn_save)

    # Save summary
    report = {'test_metrics': {'rmse': rmse(preds,trues), 'mae': mae(preds,trues), 'mape': mape(preds,trues)}, 'model': best_path}
    with open("outputs/report.json", "w") as f:
        json.dump(report, f, indent=2)

    print("All outputs saved in ./outputs")

if __name__ == "__main__":
    main()

Data shapes: (5000, 3) (5000, 2)
Training started...

Epoch 1/12 | Train Loss: 0.255097 | Val RMSE: 207.547409
Epoch 2/12 | Train Loss: 0.036009 | Val RMSE: 169.779144
Epoch 3/12 | Train Loss: 0.019264 | Val RMSE: 148.421555
Epoch 4/12 | Train Loss: 0.014420 | Val RMSE: 162.950531
Epoch 5/12 | Train Loss: 0.011359 | Val RMSE: 153.331711
Epoch 6/12 | Train Loss: 0.010027 | Val RMSE: 162.299423
Epoch 7/12 | Train Loss: 0.009027 | Val RMSE: 159.018921
Epoch 8/12 | Train Loss: 0.008007 | Val RMSE: 125.531281
Epoch 9/12 | Train Loss: 0.007458 | Val RMSE: 143.008270
Epoch 10/12 | Train Loss: 0.006504 | Val RMSE: 159.274399
Epoch 11/12 | Train Loss: 0.006045 | Val RMSE: 153.741272
Epoch 12/12 | Train Loss: 0.005977 | Val RMSE: 173.187363

Training complete! Best model saved to: outputs/best_model.pt
Test RMSE: 422.08734130859375 MAE: 402.9371337890625 MAPE: 26.015466451644897
Saved attention plot to outputs/dec_cross_attn.png
All outputs saved in ./outputs
